In [6]:
from abc import ABC, abstractmethod
from dataclasses import dataclass


# Model Sederhana
@dataclass
class Order:
    customer_name: str
    total_price: float
    status: str = "open"


# === KODE BURUK (SEBELUM REFACTOR) ===
class OrderManager:  # Melanggar SRP, OCP, DIP
    def process_checkout(self, order: Order, payment_method: str):
        print(f"Memulai checkout untuk {order.customer_name}...")

        # LOGIKA PEMBAYARAN (Pelanggaran OCP/DIP)
        if payment_method == "credit_card":
            # Logika detail implementasi hardcoded di sini
            print("Processing Credit Card...")
        elif payment_method == "bank_transfer":
            # Logika detail implementasi hardcoded di sini
            print("Processing Bank Transfer...")
        else:
            print("Metode tidak valid.")
            return False

        # LOGIKA NOTIFIKASI (Pelanggaran SRP)
        print(f"Mengirim notifikasi ke {order.customer_name}...")
        order.status = "paid"
        return True
# --- ABSTRAKSI (KONTRAK UNTUK OCP/DIP) ---
class IPaymentProcessor(ABC):
    """Kontrak: Semua prosesor pembayaran harus punya method 'process'."""
    @abstractmethod
    def process(self, order: Order) -> bool:
        pass


class INotificationService(ABC):
    """Kontrak: Semua layanan notifikasi harus punya method 'send'."""
    @abstractmethod
    def send(self, order: Order):
        pass


# --- IMPLEMENTASI KONKRIT (Plug-in) ---
class CreditCardProcessor(IPaymentProcessor):
    def process(self, order: Order) -> bool:
        print("Payment: Memproses Kartu Kredit.")
        return True


class EmailNotifier(INotificationService):
    def send(self, order: Order):
        print(f"Notif: Mengirim email konfirmasi ke {order.customer_name}.")


# --- KELAS KOORDINATOR (SRP & DIP) ---
class CheckoutService:  # Tanggung jawab tunggal: Mengkoordinasi Checkout
    def __init__(self, payment_processor: IPaymentProcessor, notifier: INotificationService):
        # Dependency Injection (DIP): Bergantung pada Abstraksi, bukan Konkrit
        self.payment_processor = payment_processor
        self.notifier = notifier

    def run_checkout(self, order: Order):
        payment_success = self.payment_processor.process(order)  # Delegasi 1

        if payment_success:
            order.status = "paid"
            self.notifier.send(order)  # Delegasi 2
            print("Checkout Sukses.")
            return True
        return False
# --- PROGRAM UTAMA ---
# Setup Dependencies
andi_order = Order("Andi", 500000)
email_service = EmailNotifier()

# 1. Inject implementasi Credit Card
cc_processor = CreditCardProcessor()
checkout_cc = CheckoutService(payment_processor=cc_processor, notifier=email_service)
print("--- Skenario 1: Credit Card ---")
checkout_cc.run_checkout(andi_order)

# 2. Pembuktian OCP: Menambah Metode Pembayaran QRIS (Tanpa Mengubah CheckoutService)
class QrisProcessor(IPaymentProcessor):
    def process(self, order: Order) -> bool:
        print("Payment: Memproses QRIS.")
        return True

budi_order = Order("Budi", 100000)
qris_processor = QrisProcessor()

# Inject implementasi QRIS yang baru dibuat
checkout_qris = CheckoutService(payment_processor=qris_processor, notifier=email_service)
print("\n--- Skenario 2: Pembuktian OCP (QRIS) ---")
checkout_qris.run_checkout(budi_order)



--- Skenario 1: Credit Card ---
Payment: Memproses Kartu Kredit.
Notif: Mengirim email konfirmasi ke Andi.
Checkout Sukses.

--- Skenario 2: Pembuktian OCP (QRIS) ---
Payment: Memproses QRIS.
Notif: Mengirim email konfirmasi ke Budi.
Checkout Sukses.


True

In [7]:
import logging
from abc import ABC, abstractmethod
from dataclasses import dataclass

# --- KONFIGURASI LOGGING ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
LOGGER = logging.getLogger()

# --- DATA MODEL ---
@dataclass
class Athlete:
    """
    Data peserta lomba olahraga.
    """
    name: str
    age: int
    medical_clearance: bool
    training_sessions: int


# --- ABSTRAKSI VALIDASI ---
class IValidationRule(ABC):
    @abstractmethod
    def validate(self, athlete: Athlete) -> bool:
        pass


# --- IMPLEMENTASI RULE ---

class AgeLimitRule(IValidationRule):
    """Usia peserta harus antara 17–25 tahun."""

    def validate(self, athlete: Athlete) -> bool:
        if not (17 <= athlete.age <= 25):
            LOGGER.warning(f"Validasi Gagal: {athlete.name} berusia {athlete.age} (Batas 17–25).")
            return False
        return True


class MedicalCheckRule(IValidationRule):
    """Peserta harus memiliki surat keterangan sehat."""

    def validate(self, athlete: Athlete) -> bool:
        if not athlete.medical_clearance:
            LOGGER.warning(f"Validasi Gagal: {athlete.name} tidak memiliki surat sehat.")
            return False
        return True


class TrainingCertificateRule(IValidationRule):
    """Peserta harus minimal mengikuti 4 sesi latihan."""

    def validate(self, athlete: Athlete) -> bool:
        if athlete.training_sessions < 4:
            LOGGER.warning(
                f"Validasi Gagal: {athlete.name} baru mengikuti {athlete.training_sessions} sesi latihan (Min 4)."
            )
            return False
        return True


# --- CHALLENGE OCP ---
class TeamAvailabilityRule(IValidationRule):
    """Cek apakah kuota dalam tim masih tersedia."""

    def validate(self, athlete: Athlete) -> bool:
        LOGGER.info(f"Info: Mengecek kuota tim untuk {athlete.name}... Kuota masih tersedia.")
        return True


# ---(SRP + DIP + OCP) ---
class RegistrationService:
    def __init__(self, rules: list[IValidationRule]):
        self.rules = rules

    def register_athlete(self, athlete: Athlete) -> bool:
        LOGGER.info(f"\n--- Memulai Validasi Pendaftaran Lomba: {athlete.name} ---")

        for rule in self.rules:
            if not rule.validate(athlete):
                LOGGER.info("Hasil Akhir: Pendaftaran Ditolak.\n")
                return False

        LOGGER.info("Hasil Akhir: Pendaftaran Diterima.\n")
        return True


# --- EKSEKUSI UTAMA ---
if __name__ == "__main__":
    
    # Peserta valid
    peserta1 = Athlete("Andi", 20, True, 5)

    rules = [
        AgeLimitRule(),
        MedicalCheckRule(),
        TrainingCertificateRule(),
        TeamAvailabilityRule(),  # OCP rule baru
    ]

    service = RegistrationService(rules)
    service.register_athlete(peserta1)

    # Peserta dengan banyak pelanggaran
    peserta2 = Athlete("Rudi", 27, False, 2)
    service.register_athlete(peserta2)


2025-12-06 02:35:26 INFO - 
--- Memulai Validasi Pendaftaran Lomba: Andi ---
2025-12-06 02:35:26 INFO - Info: Mengecek kuota tim untuk Andi... Kuota masih tersedia.
2025-12-06 02:35:26 INFO - Hasil Akhir: Pendaftaran Diterima.

2025-12-06 02:35:26 INFO - 
--- Memulai Validasi Pendaftaran Lomba: Rudi ---
2025-12-06 02:35:26 WARNING - Validasi Gagal: Rudi berusia 27 (Batas 17–25).
2025-12-06 02:35:26 INFO - Hasil Akhir: Pendaftaran Ditolak.

